In [0]:
%sql
DROP TABLE IF EXISTS cpg_industry.gold_supply.supply_b2c_kpi;
DROP TABLE IF EXISTS cpg_industry.gold_supply.supply_b2c_monthly_purchase_summary;
DROP TABLE IF EXISTS cpg_industry.gold_supply.supply_b2c_purchase_summary_by_location;
DROP TABLE IF EXISTS cpg_industry.gold_supply.supply_b2c_category_quantity_ordered;
DROP TABLE IF EXISTS cpg_industry.gold_supply.supply_b2c_order_status_summary;
DROP TABLE IF EXISTS cpg_industry.gold_supply.supply_top_inventory_products;
DROP TABLE IF EXISTS cpg_industry.gold_supply.supply_inventory_status_summary;
DROP TABLE IF EXISTS cpg_industry.gold_supply.supply_low_stock_products;
DROP TABLE IF EXISTS cpg_industry.gold_supply.supply_location_inventory_quantity;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS cpg_industry.gold_supply;

In [0]:
%sql
CREATE TABLE cpg_industry.gold_supply.supply_b2c_kpi AS
WITH base_data AS (
    SELECT
        i.product_id,
        i.quantity_on_hand,
        p.retail_price,
        coi.total_price,
        coi.quantity,
        o.order_status,
        o.order_date,
        co.order_id AS co_order_id
    FROM cpg_industry.silver.cpg_inventory i
    JOIN cpg_industry.silver.cpg_product p 
        ON i.product_id = p.product_id
    JOIN cpg_industry.silver.cpg_consumer_order_items coi 
        ON p.product_id = coi.product_id
    JOIN cpg_industry.silver.cpg_consumer_order o 
        ON coi.order_id = o.order_id
    JOIN cpg_industry.silver.cpg_consumer_order co 
        ON o.order_id = co.order_id
),
agg_level1 AS (
    SELECT
        CURRENT_DATE() AS kpi_date,
        SUM(quantity_on_hand * retail_price) AS inventory_value,
        SUM(total_price) AS total_sales_value,
        SUM(quantity) AS total_quantity_sold,
        AVG(quantity_on_hand) AS avg_quantity_on_hand,
        COUNT(DISTINCT co_order_id) AS total_orders,
        SUM(CASE WHEN order_status = 'Returned' THEN 1 ELSE 0 END) AS backorder_count,
        COUNT(*) AS total_order_lines,
        AVG(DATEDIFF(CURRENT_DATE(), order_date)) AS avg_days_sales_inventory
    FROM base_data
)
SELECT
    kpi_date,
    ROUND(inventory_value / NULLIF(total_sales_value, 0), 2) AS stock_to_sales_ratio,
    ROUND(total_quantity_sold / NULLIF(avg_quantity_on_hand, 0), 2) AS inventory_turnover_rate,
    ROUND(avg_quantity_on_hand / NULLIF(total_quantity_sold / NULLIF(total_orders, 0), 0), 2) AS weeks_on_hand,
    ROUND(backorder_count * 100.0 / NULLIF(total_order_lines, 0), 2) AS backorder_rate_pct,
    ROUND(avg_days_sales_inventory, 2) AS days_sales_in_inventory
FROM agg_level1;

In [0]:
%sql
CREATE TABLE cpg_industry.gold_supply.supply_b2c_monthly_purchase_summary AS
SELECT
    DATE_TRUNC('month', o.order_date) AS order_month,
    ROUND(SUM(coi.total_price),2) AS total_purchase_amount
FROM cpg_industry.silver.cpg_consumer_order_items coi
JOIN cpg_industry.silver.cpg_consumer_order o ON coi.order_id = o.order_id
GROUP BY DATE_TRUNC('month', o.order_date)
ORDER BY order_month;

In [0]:
%sql
CREATE TABLE cpg_industry.gold_supply.supply_b2c_purchase_summary_by_location AS
SELECT 
    c.state,
    c.country,
    ROUND(SUM(o.total_amount), 2) AS total_purchase_value
FROM cpg_industry.silver.cpg_consumer c
JOIN cpg_industry.silver.cpg_consumer_order o
    ON c.consumer_id = o.consumer_id
GROUP BY c.state, c.country
ORDER BY total_purchase_value DESC;

In [0]:
%sql
CREATE TABLE cpg_industry.gold_supply.supply_b2c_category_quantity_ordered AS
SELECT 
    pr.category,
    SUM(coi.quantity) AS total_quantity_ordered
FROM cpg_industry.silver.cpg_consumer_order_items coi
JOIN cpg_industry.silver.cpg_product pr
    ON coi.product_id = pr.product_id
GROUP BY pr.category
ORDER BY total_quantity_ordered DESC;

In [0]:
%sql
CREATE TABLE cpg_industry.gold_supply.supply_top_inventory_products AS
SELECT 
    pr.product_name,
    inv.quantity_on_hand
FROM cpg_industry.silver.cpg_inventory inv
JOIN cpg_industry.silver.cpg_product pr
    ON inv.product_id = pr.product_id
ORDER BY inv.quantity_on_hand DESC
LIMIT 10;

In [0]:
%sql
CREATE TABLE cpg_industry.gold_supply.supply_low_stock_products AS
SELECT p.product_name, inv.quantity_on_hand, inv.reorder_level
FROM cpg_industry.silver.cpg_inventory inv
JOIN cpg_industry.silver.cpg_product p ON inv.product_id = p.product_id
WHERE inv.quantity_on_hand <= inv.reorder_level
ORDER BY inv.quantity_on_hand ASC;

In [0]:
%sql
CREATE TABLE cpg_industry.gold_supply.supply_inventory_status_summary AS
SELECT 
    inventory_status,
    COUNT(*) AS count
FROM cpg_industry.silver.cpg_inventory
GROUP BY inventory_status;

In [0]:
%sql
CREATE TABLE cpg_industry.gold_supply.supply_b2c_order_status_summary AS
SELECT 
    order_status,
    COUNT(*) AS count
FROM cpg_industry.silver.cpg_consumer_order
GROUP BY order_status;

In [0]:
%sql
CREATE TABLE cpg_industry.gold_supply.supply_location_inventory_quantity AS
SELECT 
    location_name,
    state,
    country,
    SUM(quantity_on_hand) AS total_quantity
FROM cpg_industry.silver.cpg_inventory
GROUP BY country, state, location_name
ORDER BY total_quantity DESC;